# Projeto 02

## Importação das bibliotecas

In [139]:
import os
import pandas as pd
import bibtexparser
import yaml

## Carregando arquivo de configuração YAML

In [156]:
with open(r"G:\Meu Drive\MBA - Data Engeneering\Python for Data Engineer\Trabalho-part2\Projeto 2\Config\config_final.yaml", "r") as f:
    config = yaml.load(f, Loader=yaml.FullLoader)
    print("Read successful")
print(config)

Read successful
{'type_save': 'json, csv, yaml', 'filter_title': None, 'filter_year': None, 'csv_dest': 'G:\\Meu Drive\\MBA - Data Engeneering\\Python for Data Engineer\\Trabalho-part2\\Projeto 2\\Export\\export_final.csv', 'json_dest': 'G:\\Meu Drive\\MBA - Data Engeneering\\Python for Data Engineer\\Trabalho-part2\\Projeto 2\\Export\\export_final.json', 'yaml_dest': 'G:\\Meu Drive\\MBA - Data Engeneering\\Python for Data Engineer\\Trabalho-part2\\Projeto 2\\Export\\export_final.yaml'}


## Leitura dos arquivos CSV

### Arquivo JCS

In [141]:
#Leitura e padronização do arquivo JCR
df_jcs = pd.read_csv(r'G:\Meu Drive\MBA - Data Engeneering\Python for Data Engineer\Trabalho-part2\Projeto 2\Source\jcs_2020.csv', sep=';')

#Guardando os dados brutos
df_jcs_raw = df_jcs.copy()

#Dropando colunas não utilizadas
df_jcs.drop(["Rank", "Unnamed: 3", "Unnamed: 6", "Unnamed: 7"], axis=1, inplace=True)

#Renomeando coluna
df_jcs.rename(columns={'Full Journal Title':'Title'}, inplace=True)

#Removendo as linhas duplicadas
df_jcs.drop_duplicates(keep=False,inplace=True)

#Criando uma coluna para o join
df_jcs['Title_join'] = df_jcs.Title
df_jcs['Title_join'] = df_jcs['Title_join'].str.upper()

df_jcs.head(2)

,Title,Total Cites,Journal Impact Factor,Eigenfactor Score,Title_join
0,CA-A CANCER JOURNAL FOR CLINICIANS,"55,868",508.702,0.10514,CA-A CANCER JOURNAL FOR CLINICIANS
1,NATURE REVIEWS MOLECULAR CELL BIOLOGY,"58,477",94.444,0.07548,NATURE REVIEWS MOLECULAR CELL BIOLOGY


### Arquivo SCIMAGO

In [142]:
#Leitura e padronização do arquivo SCIMAGO
df_scimago = pd.read_csv(
    r'G:\Meu Drive\MBA - Data Engeneering\Python for Data Engineer\Trabalho-part2\Projeto 2\Source\scimagojr 2020.csv', 
    sep=';', quotechar='"', low_memory=False)

#Guardando os dados brutos
df_scimago_raw = df_scimago.copy()

#Dropando colunas não utilizadas
df_scimago.drop(["Rank"], axis=1, inplace=True)

#Renomeando coluna
df_scimago.rename(columns={'Issn':'issn'}, inplace=True)

#Capturando os primeiros 8 caracteres do issn
df_scimago['issn'] = df_scimago['issn'].str[0:8]

#Removendo as linhas duplicadas
df_scimago.drop_duplicates(keep=False,inplace=True)

#Criando uma coluna
df_scimago['Title_join'] = df_scimago.Title
df_scimago['Title_join'] = df_scimago['Title_join'].str.upper()

df_scimago.head(2)

,Sourceid,Title,Type,issn,SJR,SJR Best Quartile,H index,Total Docs. (2020),Total Docs. (3years),Total Refs.,Total Cites (3years),Citable Docs. (3years),Cites / Doc. (2years),Ref. / Doc.,Country,Region,Publisher,Coverage,Categories,Title_join
0,28773,Ca-A Cancer Journal for Clinicians,journal,15424863,"62,937",Q1,168,47,119,3452,15499,80,"126,34","73,45",United States,Northern America,Wiley-Blackwell,1950-2020,Hematology (Q1); Oncology (Q1),CA-A CANCER JOURNAL FOR CLINICIANS
1,19434,MMWR Recommendations and Reports,journal,10575987,"40,949",Q1,143,10,9,1292,492,9,"50,00","129,20",United States,Northern America,Centers for Disease Control and Prevention (CDC),1990-2020,Epidemiology (Q1); Health Information Manageme...,MMWR RECOMMENDATIONS AND REPORTS


In [143]:
#Realizando join entre o arquivo SCIMAGO e JCS
df_join = pd.merge(df_scimago, df_jcs, on=["Title_join"], how="left")

In [144]:
#Apagando a coluna Title_y e Title_join
df_join.drop(["Title_y", "Title_join"], axis=1, inplace=True)

#Renomeando a coluna Title_x para Title
df_join.rename(columns={"Title_x":"Title"}, inplace=True)

In [145]:
df_join.head(3)

,Sourceid,Title,Type,issn,SJR,SJR Best Quartile,H index,Total Docs. (2020),Total Docs. (3years),Total Refs.,...,Cites / Doc. (2years),Ref. / Doc.,Country,Region,Publisher,Coverage,Categories,Total Cites,Journal Impact Factor,Eigenfactor Score
0,28773,Ca-A Cancer Journal for Clinicians,journal,15424863,"62,937",Q1,168,47,119,3452,...,"126,34","73,45",United States,Northern America,Wiley-Blackwell,1950-2020,Hematology (Q1); Oncology (Q1),"55,868",508.702,0.10514
1,19434,MMWR Recommendations and Reports,journal,10575987,"40,949",Q1,143,10,9,1292,...,"50,00","129,20",United States,Northern America,Centers for Disease Control and Prevention (CDC),1990-2020,Epidemiology (Q1); Health Information Manageme...,"3,288",55.857,0.01123
2,20315,Nature Reviews Molecular Cell Biology,journal,14710072,"37,461",Q1,431,115,338,8439,...,"32,83","73,38",United Kingdom,Western Europe,Nature Publishing Group,2000-2020,Cell Biology (Q1); Molecular Biology (Q1),"58,477",94.444,0.07548


## Leitura de Arquivos BibTex

### Arquivo ACM 

In [146]:
#Leitura e padronização do arquivo
with open(r"G:\Meu Drive\MBA - Data Engeneering\Python for Data Engineer\Trabalho-part2\Projeto 2\Source\ACM.bib", encoding="utf8") as bibtex_file_acm:
    bibtex_database_acm = bibtexparser.load(bibtex_file_acm)     
    
df_acm = pd.DataFrame(bibtex_database_acm.entries)

#Guardando os dados brutos
df_acm_raw = df_acm.copy()

#Renomeando coluna
df_acm.rename(columns={'ENTRYTYPE':'type_publication'}, inplace=True)

#Removendo '-' da coluna issn
df_acm['issn'].replace('-', '', regex=True, inplace=True)


df_acm.head(2)

,series,location,keywords,numpages,pages,booktitle,abstract,doi,url,address,...,articleno,month,journal,issn,number,volume,issue_date,note,edition,editor
0,BDE 2019,"Hong Kong, Hong Kong","Sentiment analysis, Big data quality metrics, ...",8,36–43,Proceedings of the 2019 International Conferen...,"In a world increasingly connected, and in whic...",10.1145/3341620.3341629,https://doi.org/10.1145/3341620.3341629,"New York, NY, USA",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,SITA'20,"Rabat, Morocco","Data Quality evaluation, Data Quality, Quality...",6,NaN,Proceedings of the 13th International Conferen...,"In recent years, as more and more data sources...",10.1145/3419604.3419803,https://doi.org/10.1145/3419604.3419803,"New York, NY, USA",...,16,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Arquivo IEE 

In [147]:
#Leitura e padronização do arquivo
with open(r"G:\Meu Drive\MBA - Data Engeneering\Python for Data Engineer\Trabalho-part2\Projeto 2\Source\IEE.bib", encoding="utf8") as bibtex_file_iee:
    bibtex_database_iee = bibtexparser.load(bibtex_file_iee)     
    
df_iee = pd.DataFrame(bibtex_database_iee.entries)

#Guardando os dados brutos
df_iee_raw = df_iee.copy()

#Renomeando coluna
df_iee.rename(columns={'ENTRYTYPE':'type_publication'}, inplace=True)

#Removendo '-' da coluna issn
df_iee['issn'].replace('-', '', regex=True, inplace=True)


df_iee.head(2)

,month,issn,doi,keywords,abstract,pages,number,volume,year,title,booktitle,author,type_publication,ID,journal,url,isbn,publisher
0,June,,10.1109/BigDataCongress.2017.73,Big Data;Optimization;Data models;Quality asse...,"In the Big Data Era, data is the core for any ...",498-501,,,2017,Big Data Pre-Processing: Closing the Data Qual...,2017 IEEE International Congress on Big Data (...,"Taleb, Ikbal and Serhani, Mohamed Adel",inproceedings,8029366,NaN,NaN,NaN,NaN
1,Nov,,10.1109/WISA.2017.29,Big Data;Data integrity;Power grids;History;Re...,Since a low-quality data may influence the eff...,289-292,,,2017,A Big Data Framework for Electric Power Data Q...,2017 14th Web Information Systems and Applicat...,"Liu, He and Huang, Fupeng and Li, Han and Liu,...",inproceedings,8332632,NaN,NaN,NaN,NaN


### Arquivo ScienceDirect

In [148]:
#Leitura e padronização do arquivo
with open(r"G:\Meu Drive\MBA - Data Engeneering\Python for Data Engineer\Trabalho-part2\Projeto 2\Source\ScienceDirect.bib", encoding="utf8") as bibtex_file_ScienceDirect:
    bibtex_database_ScienceDirect = bibtexparser.load(bibtex_file_ScienceDirect)     
    
df_ScienceDirect = pd.DataFrame(bibtex_database_ScienceDirect.entries)

#Guardando os dados brutos
df_ScienceDirect_raw = df_ScienceDirect.copy()

#Renomeando coluna
df_ScienceDirect.rename(columns={'ENTRYTYPE':'type_publication'}, inplace=True)

#Removendo '-' da coluna issn
df_ScienceDirect['issn'].replace('-', '', regex=True, inplace=True)


df_ScienceDirect.head(2)

,abstract,keywords,author,url,doi,issn,note,year,pages,volume,...,type_publication,ID,number,booktitle,publisher,series,editor,isbn,address,edition
0,"Beyond the hype of Big Data, something within ...","Data Quality, Big Data, Measurement, Quality-i...",Jorge Merino and Ismael Caballero and Bibiano ...,https://www.sciencedirect.com/science/article/...,https://doi.org/10.1016/j.future.2015.11.024,0167739X,Modeling and Management for Big Data Analytics...,2016,123-130,63,...,article,MERINO2016123,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,Xintao Liu and Min Chen and Christophe Claramu...,https://www.sciencedirect.com/science/article/...,https://doi.org/10.1016/j.xinn.2022.100279,26666758,NaN,2022,100279,3,...,article,LIU2022100279,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [149]:
#Concatenando os dataframes acm, iee e ScienceDirect
df_bib = pd.concat([df_acm, df_iee, df_ScienceDirect])

In [150]:
#Efetuando o join com a coluna issn
df_final = pd.merge(df_join, df_bib, on=["issn"], how="inner")

#Removendo as linhas duplicadas
df_final.drop_duplicates(keep=False,inplace=True)

In [153]:
print(config["filter_title"])
print(config["filter_year"])

None
2019


In [157]:
export = (config["type_save"]).upper()
filter_title = (config["filter_title"])
filter_year = (config["filter_year"])

if "CSV" in export or "JSON" in export or "YAML" in export:
    ##CSV
    if "CSV" in export:
        if filter_year is not None and filter_title is not None:
            filter_title = filter_title.upper()
            df_final[(df_final['year'] == str(filter_year)) & (df_final['Title'].str.upper().str.contains(filter_title))].to_csv(config["csv_dest"], sep='|', encoding='utf-8', header='true')
            print("Export para csv realizado com filtro no campo year e Title feito com sucesso!")
                        
        elif filter_year is not None:
            df_final[df_final['year'] == str(filter_year)].to_csv(config["csv_dest"], sep='|', encoding='utf-8', header='true')
            print("Export para csv realizado com filtro no campo year feito com sucesso!")
            
        elif filter_title is not None:
            filter_title = filter_title.upper()
            df_final[df_final['Title'].str.upper().str.contains(filter_title)].to_csv(config["csv_dest"], sep='|', encoding='utf-8', header='true')
            print("Export para csv realizado com filtro no campo Title feito com sucesso!")

        else:
            df_final.to_csv(config["csv_dest"], sep='|', encoding='utf-8', header='true')
            print("Export para csv realizado com sucesso - sem filtro!")
    
    ##JSON
    if "JSON" in export:        
        if filter_year is not None and filter_title is not None:
            filter_title = filter_title.upper()
            df_final[(df_final['year'] == str(filter_year)) & (df_final['Title'].str.upper().str.contains(filter_title))].to_json(config["json_dest"], orient='records')
            print("Export para json realizado com filtro no campo year e Title feito com sucesso!")
            
        elif filter_year is not None:
            df_final[df_final['year'] == str(filter_year)].to_json(config["json_dest"], orient='records')
            print("Export para json realizado com filtro no campo year feito com sucesso!")
            
        elif filter_title is not None:
            filter_title = filter_title.upper()
            df_final[df_final['Title'].str.upper().str.contains(filter_title)].to_json(config["json_dest"], orient='records')
            print("Export para json realizado com filtro no campo Title feito com sucesso!")

        else:
            df_final.to_json(config["json_dest"], orient='records')
            print("Export para json realizado com sucesso - sem filtro!")
     
    ##YAML        
    if "YAML" in export: 
        if filter_year is not None and filter_title is not None:
            filter_title = filter_title.upper()
            export_yaml = yaml.dump(
                df_final[(df_final['year'] == str(filter_year)) & (df_final['Title'].str.upper().str.contains(filter_title))].to_dict(orient='records'),
                    sort_keys=False, default_flow_style=None)
            
            with open(config["yaml_dest"], 'w') as f:
                f.write(export_yaml)
            print("Export para yaml realizado com filtro no campo year e Title feito com sucesso!")
            
        elif filter_year is not None:
            export_yaml = yaml.dump(
                df_final[df_final['year'] == str(filter_year)].to_dict(orient='records'),
                    sort_keys=False, default_flow_style=None)
            
            with open(config["yaml_dest"], 'w') as f:
                f.write(export_yaml)
            print("Export para json realizado com filtro no campo year feito com sucesso!")
            
        elif filter_title is not None:
            filter_title = filter_title.upper()
            export_yaml = yaml.dump(
                df_final[df_final['Title'].str.upper().str.contains(filter_title)].to_dict(orient='records'),
                    sort_keys=False, default_flow_style=None)
            
            with open(config["yaml_dest"], 'w') as f:
                f.write(export_yaml)
            print("Export para json realizado com filtro no campo Title feito com sucesso!")

        else:
            export_yaml = yaml.dump(
                df_final.to_dict(orient='records'),
                    sort_keys=False, default_flow_style=None)
            
            with open(config["yaml_dest"], 'w') as f:
                f.write(export_yaml)
            print("Export para yaml realizado com sucesso - sem filtro!")
        
        
        
##ERRO - formato de export invalido        
else:
    print(f'O formato \'{export}\' solicitado não existe!\nOs formatos disponíveis são: csv, json e yaml')

Export para csv realizado com sucesso - sem filtro!
Export para json realizado com sucesso - sem filtro!
Export para json realizado com sucesso - sem filtro!
